In [28]:
import random
import cv2
import os
import shutil
import numpy as np
from tqdm.auto import tqdm

In [ ]:
# df = pd.read_csv("trainVal.csv")
# arrUnique = df['lp'].unique()
# ret = []
# for unique in tqdm(arrUnique):
#     uniqueIndexs = df[df['lp']==unique].index
#     index = random.randint(0,uniqueIndexs.size-1)
#     img_info = df.loc[uniqueIndexs[index]]
#     ret.append(np.array([img_info[1], img_info[2]]))
#     
# ret = np.array(ret)
# np.save("hlp-locations.npy", ret)

In [29]:
DIR = "../Datasets/hlp"
DIR_PLACAS = "../Datasets/hlp/plates"
DIR_TREINO = "../Datasets/hlp/train"
DIR_TESTE = "../Datasets/hlp/test"
CLASSES = 10

ROW_SIZE = 26
COL_SIZE = 13

In [ ]:
arr = np.load("hlp-locations.npy")


In [ ]:
'''
    Separa placas com 7 digitos e que os 4 ultimos sejam numerais.
    arr -> [ [img_path, LP] ... ]
    arr_sem_letra -> [ [img_path, LP] ... ] 
'''
arr_sem_letra = []
for img in tqdm(arr):
    if len(img[1]) != 7:
        continue
    if img[1][-4:].isnumeric() is False:
        continue
    arr_sem_letra.append(img)
arr_sem_letra = np.array(arr_sem_letra)


In [ ]:
np.save("hlp-locations-final.npy", arr_sem_letra)

In [ ]:
arr = np.load("hlp-locations-final.npy")


In [ ]:
'''
    Agrupa as placas que possuem outras placas com os mesmos 4 ultrimos digitos.
    arr -> [ [img_path, LP] ... ]
    unicos_entre_si -> { LP[-4:] : [ [img_path, LP] ...] ... }
    
'''
unicos_entre_si = {}
for img_info in tqdm(arr):
    if img_info[1][-4:] in unicos_entre_si:
        unicos_entre_si[img_info[1][-4:]].append(img_info)
    else:
        unicos_entre_si[img_info[1][-4:]] = [img_info]

In [ ]:
'''
    Monta o numpy.array com o path e a license plate (LP) de cada placa em unicos_entre_si.
    Para os casos de chaves que possuem mais de um  valor, escolhe aleatoriamente apenas 1 de cada.
    i.e:
    {'4698' : [
                ['s01_l01/2_7.png','7C24698'],
                ['s01_l02/479_6.png','6B44698']
                ] } -seleciona randomicamente-> ['s01_l02/479_6.png','6B44698']
    {'2347' : [
                ['s01_l01/4_17.png', '5AR2347']
                ] } ----seleciona o unico-----> ['s01_l01/4_17.png', '5AR2347']
'''
arr_unico_entre_si = []
for img in tqdm(unicos_entre_si):
    index = random.randint(0, len(unicos_entre_si[img])-1)
    arr_unico_entre_si.append(unicos_entre_si[img][index])
arr_unico_entre_si = np.array(arr_unico_entre_si)

In [ ]:
np.save("hlp-locations-final.npy", arr_unico_entre_si)


In [ ]:
arr = np.load('hlp-locations-final.npy')

In [ ]:

'''
    Copia as imagens das placas separadas em arr_unico_entre_si para uma pasta (Treino).
'''
for img in tqdm(arr):
    try:
        shutil.copy(os.path.join(DIR, img[0]), os.path.join(DIR_PLACAS, f"{img[1]}.png"))
    except shutil.Error as err:
        print(err)


In [ ]:
'''
    Segmentação dos números da placa.
'''
for  img_name in tqdm(os.listdir(DIR_PLACAS)):
    image = cv2.imread(os.path.join(DIR_PLACAS, img_name))
    
    # grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    for limiar in range(50, 200):
        # binary
        _, thresh = cv2.threshold(gray, limiar, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('threshold', thresh)
    
        # dilation
        kernel = np.ones((1, 1), np.uint8)
        img_dilation = cv2.dilate(thresh, kernel, iterations=1)
        #cv2.imshow('dilated', img_dilation)
    
    
        ctrs, _ = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
        # sort contours
        sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        roiList = []
    
        for ctr in sorted_ctrs:
            # Get bounding box
            x, y, w, h = cv2.boundingRect(ctr)
    
            # Getting ROI
            roi = image[y:y + h, x:x + w]
            row, column, _ = roi.shape
    
            if (20 <= row <= 30) and (10 <= column <= 20):
                roiList.append(roi)
    
        if len(roiList) == 4:
            name = list(img_name[:4])
            
            for i, roi in enumerate(roiList):
                cv2.imwrite(os.path.join(DIR_TREINO, f"{name[i]}_{img_name[:4]}_{i}.png"), roi)
            break

In [ ]:
"""
    Separa imagens aleatorias para teste
"""

folders = { '0': 41, '1': 34, '2': 40,
			'3': 34, '4': 41, '5': 31,
			'6': 36, '7': 36, '8': 38,
			'9': 34
			}

for folder in folders:
	files = os.listdir(os.path.join(DIR_TREINO, folder))

	selected = random.sample(files, folders[folder])

	for file in selected: 
		shutil.move(os.path.join(DIR_TREINO, folder, file), os.path.join(DIR_TESTE, folder, file))

In [30]:
"""
    Prepara o Vetor de treino e de Validação (70% de treino e 30% de validação)
    training_data -> [ [ IMG_MATRIX, IMG_LABEL ], ... ]
    validating_data -> [ [ IMG_MATRIX, IMG_LABEL ], ... ]
"""
folders = { '0': 233, '1': 196, '2': 226,
			'3': 196, '4': 237, '5': 177,
			'6': 206, '7': 208, '8': 217,
			'9': 193
			}

validating_data = []
training_data = []

for folder in folders:
    files = os.listdir(os.path.join(DIR_TREINO, folder))
    selected = random.sample(files, folders[folder])
    
    # Remove arquivos selecionados do vetor 'files'
    for file in selected:
        files.remove(file)
        
    # Validation Data
    for img_name in tqdm(selected):
        img = cv2.imread(os.path.join(DIR_TREINO, folder, img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (COL_SIZE, ROW_SIZE))
        
        validating_data.append([img, folder])
    
    
    # Validation Data
    for img_name in tqdm(files):
        img = cv2.imread(os.path.join(DIR_TREINO, folder, img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (COL_SIZE, ROW_SIZE))
        
        training_data.append([img, folder])
        
validating_data = np.array(validating_data)
training_data = np.array(training_data)
np.save(f"hlp-dataset-{training_data.shape[0]}-({ROW_SIZE}, {COL_SIZE})-train.npy", training_data)
np.save(f"hlp-dataset-{validating_data.shape[0]}-({ROW_SIZE}, {COL_SIZE})-validate.npy", validating_data)

del training_data
del validating_data

In [31]:
"""
    Prepara o Vetor de teste
    testing_data -> [ [ IMG_MATRIX, IMG_LABEL ], ... ]
"""

testing_data = []
for folder in range(10):
    folder = str(folder)
    for img_name in tqdm(os.listdir(os.path.join(DIR_TESTE, folder))):
        img = cv2.imread(os.path.join(DIR_TESTE, folder, img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (COL_SIZE, ROW_SIZE))
        
        testing_data.append([img, folder])
        
testing_data = np.array(testing_data)
np.save(f"hlp-dataset-{testing_data.shape[0]}-({ROW_SIZE}, {COL_SIZE})-test.npy", testing_data)
del testing_data


In [27]:
%reset -f